In [1]:
from abc import ABC, abstractmethod


class HashMap:

    class Entry:
        def __init__(self, key, value=None):
            self.key = key
            self.value = value

        def get_key(self):
            return self.key

        def get_value(self):
            return self.value

        def __eq__(self, other):
            return self.key == other.key


    def __init__(self, bucket_num=64):
        self.size = 0
        self.count_buckets = bucket_num
        self.buckets = [[] for i in range(self.count_buckets)]

    def _find_in_buckets(self, key):
        index = self._get_index(key)
        x = self.Entry(key)
        if x in self.buckets[index]:
            return (index, self.buckets[index].index(x))
        return (index, -1)

    def get(self, key, default_value=None):
        index, key_id = self._find_in_buckets(key)
        return self.buckets[index][key_id].get_value() if key_id != -1 else default_value

    def put(self, key, value):
        index, key_id = self._find_in_buckets(key)
        if key_id != -1:
            self.buckets[index][key_id].value = value
            return False
        self.buckets[index].append(self.Entry(key, value))
        self.size += 1
        return True

    def __len__(self):
        return self.size

    def _get_hash(self, key):
        return hash(key)

    def _get_index(self, key):
        return self._get_hash(key) % self.count_buckets

    def values(self):
        return ValuesView(self)

    def keys(self):
        return KeysView(self)

    def items(self):
        return ItemsView(self)

    def __str__(self):
        res = ", ".join("({!r} -> {!r})".format(x.get_key(), x.get_value()) for x in self.items())
        return "{" + res + "}"


class View(ABC):
    def __init__(self, hash_map):
        self.hm = hash_map
        self.bucket_id = 0
        self.id = 0

    def __iter__(self):
        return self

    @abstractmethod
    def return_value(self, x):
        pass

    def __next__(self):
        while self.bucket_id < self.hm.count_buckets:
            while self.id < len(self.hm.buckets[self.bucket_id]):
                self.id += 1
                return self.return_value(self.hm.buckets[self.bucket_id][self.id - 1])
            self.bucket_id += 1
            self.id = 0
        raise StopIteration


class ValuesView(View):
    def return_value(self, x):
        return x.value


class KeysView(View):
    def return_value(self, x):
        return x.key


class ItemsView(View):
    def return_value(self, x):
        return x

In [2]:
class HashSet(HashMap):
    def has(self, key):
        return super().get(key, False)

    def put(self, key):
        return super().put(key, True)

    def __len__(self):
        return super().__len__()

    def values(self):
        return super().keys()
    
    def __str__(self):
        res = ", ".join("{!r}".format(x.get_key()) for x in self.items())
        return "{" + res + "}"